In [1]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install apscheduler

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: apscheduler in c:\users\krist\appdata\local\programs\python\python311\lib\site-packages (3.10.4)




[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install oandapyV20

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install oanda_candles

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import pandas_ta as ta
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [6]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    #Get range of candles in window
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    #Check if all EMAA_fast vals are < EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0

In [7]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles) == 2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #anddf.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles) == 1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
          return 1
    return 0

Connecting to Market

In [10]:
access_token = ''
accountID = '101-001-25962876-001'

In [11]:
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M5)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)
    print(candle)

True
<Candle: 2024-05-03 08:45 PM>
True
<Candle: 2024-05-03 08:50 PM>
True
<Candle: 2024-05-03 08:55 PM>


In [12]:
def count_opened_trades():
    client = API(access_token=access_token)
    r = trades.OpenTrades(accountID=accountID)
    client.request(r)
    return len(r.response['trades'])

In [13]:
def get_candles_frame(n):
    candles = get_candles(n)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] =float(str(candle.bid.l))
        i = i + 1
    
    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['CLose'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    dfstream["ATR"] = ta.atr(dfstream.High, dfstream.Low, dfstream.Close, length=7)
    dfstream["EMA_fast"] = ta.ema(dfstream.Close, length=30)
    dfstream["EMA_slow"] = ta.ema(dfstream.Close, length=50)
    dfstream['RSI'] = ta.rsi(dfstream.Close, length=10)
    my_bbands = ta.bbands(dfstream.Close, length=15, std=1.5)
    dfstream = dfstream.join(my_bbands)

    return dfstream

In [14]:
import sys
from datetime import datetime

def trading_job():
    dfstream = get_candles_frame(70)

    signal = total_signal(dfstream, len(dfstream)-1, 7)

    slatr = 1.1*dfstream.ATR.iloc[-1]
    TPSLRatio = 1.5
    max_spread = 16e-5

    candle = get_candles(1)[-1]
    candle_open_bid = float(str(candle.bid.o))
    candle_open_ask = float(str(candle.ask.o))
    spread = candle_open_ask-candle_open_bid

    SLBuy = candle_open_bid-slatr-spread
    SLSell = candle_open_ask+slatr+spread

    TPBuy = candle_open_ask+slatr*TPSLRatio+spread
    TPSell = candle_open_bid-slatr*TPSLRatio-spread

    client = API(access_token=access_token)

    #Sell
    if signal == 1 and count_opened_trades() == 0 and spread < max_spread:
        print("Sell Signal Found...")
        mo = MarketOrderRequest(instrument="EUR_USD", units=-3000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        ry = client.request(r)
        print(ry)
    #Buy
    elif signal == 2 and count_opened_trades() == 0 and spread < max_spread:
        print("Buy Signal FOund...")
        mo = MarketOrderRequest(instrument="EUR_USD", units=3000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        ry = client.request(r)
        print(ry)

Executing Orders

In [ ]:
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', days_of_week='mon-fri', hour='00-23', minute='1,6,11,16,21,26,31,36,41,45,51,56', timezone='America/Chicago')
scheduler.start()